In [1]:
import openpyxl as xl
import pandas as pd
import numpy as np
from scipy import stats
from typing import Sequence, Literal, Dict, Any
import math


wb = xl.load_workbook('/Users/yut0takagi/Develop/dsb_katoh/Project1 Metrics_notest (2)のコピー2.xlsx')
sheet = wb["Tot Fixation dur incl 0"]
sheet_output = wb["aoi_解析"]
sheet_stats = wb["t検定結果"]


In [2]:

def ttest_two_groups(
    group1: Sequence[float],
    group2: Sequence[float],
    *,
    equal_var: bool | None = None,
    transform: Literal['none', 'logit'] = 'none',
    clip_eps: float = 1e-6,
    return_all: bool = True
) -> Dict[str, Any]:
    """
    2群の t 検定と効果量を計算するユーティリティ関数。
    
    Parameters
    ----------
    group1, group2 : Sequence[float]
        数値リスト（list, tuple, np.array）。None/NaN は除外。
    equal_var : bool | None
        True なら Student, False なら Welch。
        None の場合は Levene 検定 (p>=0.05) なら True, そうでなければ False を自動選択。
    transform : {'none','logit'}
        'logit' を指定すると比率(0〜1)に微小クリップした上で log(p/(1-p)) へ変換して検定。
    clip_eps : float
        logit 変換時の端点クリップ閾値。0,1 を避けるため [clip_eps, 1-clip_eps] へ収める。
    return_all : bool
        True なら補助統計（分散, 標準誤差など）も返す。
    
    Returns
    -------
    result : dict
        主キー: 
          'test' ('student' or 'welch'),
          't', 'df', 'p',
          'mean1','mean2','mean_diff',
          'ci_diff' (95%CI tuple),
          'cohens_d','hedges_g','g_ci' (95%CI),
          'glass_delta',
          'levene_p'
        transform='logit' の場合は 'inverse_means' (原スケール近似) を付加。
    """
    def _clean(x):
        arr = np.array([v for v in x if v is not None], dtype=float)
        arr = arr[~np.isnan(arr)]
        return arr

    g1 = _clean(group1)
    g2 = _clean(group2)
    if g1.size < 2 or g2.size < 2:
        raise ValueError("各群には少なくとも2つ以上の有効な数値が必要です。")

    original_means = (g1.mean(), g2.mean())

    if transform == 'logit':
        g1 = np.clip(g1, clip_eps, 1 - clip_eps)
        g2 = np.clip(g2, clip_eps, 1 - clip_eps)
        g1 = np.log(g1 / (1 - g1))
        g2 = np.log(g2 / (1 - g2))

    n1, n2 = g1.size, g2.size
    mean1, mean2 = g1.mean(), g2.mean()
    var1, var2 = g1.var(ddof=1), g2.var(ddof=1)

    # Levene で等分散性判定（center='mean'）
    levene_stat, levene_p = stats.levene(g1, g2, center='mean')

    # equal_var 自動決定
    if equal_var is None:
        equal_var = (levene_p >= 0.05)

    # t 検定
    t_stat, p_val = stats.ttest_ind(g1, g2, equal_var=equal_var)

    if equal_var:
        df = n1 + n2 - 2
        test_type = 'student'
        # プール分散
        sp2 = ((n1 - 1)*var1 + (n2 - 1)*var2) / (n1 + n2 - 2)
        se_diff = math.sqrt(sp2 * (1/n1 + 1/n2))
    else:
        # Welch の自由度
        df = (var1/n1 + var2/n2)**2 / ((var1**2)/(n1**2*(n1-1)) + (var2**2)/(n2**2*(n2-1)))
        test_type = 'welch'
        se_diff = math.sqrt(var1/n1 + var2/n2)

    mean_diff = mean1 - mean2
    t_crit = stats.t.ppf(0.975, df)
    ci_diff = (mean_diff - t_crit*se_diff, mean_diff + t_crit*se_diff)

    # 効果量
    # プール標準偏差（等分散仮定用）
    sp = math.sqrt(((n1 - 1)*var1 + (n2 - 1)*var2) / (n1 + n2 - 2))
    cohens_d = (mean1 - mean2) / sp

    # Hedges g 補正
    J = 1 - 3/(4*(n1 + n2) - 9)
    hedges_g = cohens_d * J

    # g の SE (近似) & CI
    se_g = math.sqrt((n1 + n2)/(n1*n2) + (hedges_g**2)/(2*(n1 + n2 - 2)))
    g_ci = (hedges_g - t_crit*se_g, hedges_g + t_crit*se_g)

    # Glass Δ (第2群を基準：分散異質時の代替)
    glass_delta = (mean1 - mean2) / math.sqrt(var2)

    result = {
        'test': test_type,
        'equal_var_assumed': equal_var,
        't': t_stat,
        'df': df,
        'p': p_val,
        'mean1': mean1,
        'mean2': mean2,
        'mean_diff': mean_diff,
        'ci_diff': ci_diff,
        'cohens_d': cohens_d,
        'hedges_g': hedges_g,
        'g_ci': g_ci,
        'glass_delta': glass_delta,
        'n1': n1,
        'n2': n2,
        'var1': var1,
        'var2': var2,
        'levene_p': levene_p,
        'transform': transform,
    }

    if transform == 'logit':
        # 参考：逆 logit で平均（logit平均）を原スケールへ近似
        inv = lambda x: 1 / (1 + math.exp(-x))
        result['inverse_means'] = (inv(mean1), inv(mean2))

    if not return_all:
        # 最小限
        keys = ['test','t','df','p','mean1','mean2','mean_diff','ci_diff','hedges_g','g_ci']
        result = {k: result[k] for k in keys}
    return result


In [3]:
a_high = ["週に数回","月に数回"]
a_low = ["ほとんどつけない","全くつけない"]
pjt_count = 0
for row in range(1,175):
    if sheet.cell(row,2).value in a_high:
        sheet.cell(row,1).value = 1
    elif sheet.cell(row,2).value in a_low:
        sheet.cell(row,1).value = 0
    elif len(f"{sheet.cell(row, 3).value}") > 5:
        if sheet.cell(row, 3).value[0:5] == "Total":
            sheet.cell(row -1,1).value = f"pjt:{sheet.cell(row -1, 3).value}"
    else:
        sheet.cell(row,1).value = "skip"
wb.save('/Users/yut0takagi/Develop/dsb_katoh/20250721.xlsx')

In [4]:
pjt_row = []
op_max = 1

for row in range(1,175):
    if len(f"{sheet.cell(row, 1).value}")>=1:
        if f"{sheet.cell(row, 1).value}"[0] == "p":
            pjt_row.append(row+1)
print(pjt_row)
for pjt in pjt_row:
    condit = True
    col = 5
    aoi_list = []
    
    while condit:
        aoi_list.append(sheet.cell(pjt, col).value)
        col += 1
        condit = sheet.cell(pjt, col).value != "Average"
        
    
    print(f"aoi:{aoi_list}")
    for aoi in range(len(aoi_list)):
        segment_1, segment_2 = [], []
        for row in range(pjt+1, pjt+11):
            value = sheet.cell(row,aoi + 5).value
            sum_time = wb["Time!!"].cell(row,5)
            if sheet.cell(row,1).value ==1:
                segment_1.append(value)
            else:
                segment_2.append(value)
        # output に書き出し
        for index,op_row in enumerate(range(op_max+1, op_max+6)):
            sheet_output.cell(op_row,1).value= sheet.cell(pjt-1,3).value
            sheet_output.cell(op_row,2).value = aoi_list[aoi]
            sheet_output.cell(op_row,3).value = index+1
            sheet_output.cell(op_row,4).value = segment_1[index]
            sheet_output.cell(op_row,5).value = segment_2[index]
        op_max += 5

output_row_max = 2
for row in range(1,422):
    if sheet_output.cell(row,3).value == 5:
        segment_1 = []
        segment_2 = []
        for col in range(5):
            segment_1.append(sheet_output.cell(row-4 + col,4).value)
            segment_2.append(sheet_output.cell(row-4 +col, 5).value)
        res = ttest_two_groups(segment_1, segment_2, transform='logit')
        sheet_stats.cell(output_row_max, 1).value = sheet_output.cell(row,1).value
        sheet_stats.cell(output_row_max, 2).value = sheet_output.cell(row,2).value
        for index, value in enumerate(res.values()):
            # numpy型 → Python float
            if isinstance(value, (np.floating,)):
                value = float(value)
            # タプル / リスト → "a,b" 文字列
            if isinstance(value, (tuple, list)):
                try:
                    value = ",".join(f"{float(v):.6g}" for v in value)
                except Exception:
                    value = str(value)
            sheet_stats.cell(output_row_max, index + 3).value = value
        output_row_max += 1
    
wb.save('/Users/yut0takagi/Develop/dsb_katoh/20250721.xlsx')

[2, 20, 38, 56, 74, 92, 110, 128, 146, 164]
aoi:['女all', '女アクセ無し', '女イヤリング', '女ネックレス', '男all', '男アクセ無し', '男イヤリング', '男ネックレス']
aoi:['女L', '女R', '女イヤリング\u3000顔', '女イヤリング\u3000髪型']
aoi:['女all \u3000顔', '女all ネックレス', '女all 髪型', '女allL', '女alllR']
aoi:['Rectangle', '女無し L', '女無し\u3000顔', '女無しR', '女無しネックレス']
aoi:['女ネックレス', '女ネックレス\u3000顔', '女ネックレス\u3000髪型']
aoi:['男all L', '男all R', '男all ネックレス', '男all 顔', '男all 髪型']
aoi:['男L', '男R', '男イヤリング 顔', '男イヤリング\u3000髪']
aoi:['男なし\u3000L', '男なし\u3000R', '男なし\u3000ネックレス', '男なし\u3000顔', '男なし\u3000髪型']
aoi:['男ネックレス', '男ネックレス\u3000顔', '男ネックレス 髪型']
aoi:['Rectangle', '女all', '女all \u3000顔', '女all ネックレス', '女all 髪型', '女allL', '女alllR', '女L', '女R', '女アクセ無し', '女イヤリング', '女イヤリング\u3000顔', '女イヤリング\u3000髪型', '女ネックレス', '女ネックレス', '女ネックレス\u3000顔', '女ネックレス\u3000髪型', '女無し L', '女無し\u3000顔', '女無しR', '女無しネックレス', '男all', '男all L', '男all R', '男all ネックレス', '男all 顔', '男all 髪型', '男L', '男R', '男アクセ無し', '男イヤリング', '男イヤリング 顔', '男イヤリング\u3000髪', '男なし\u3000L', '男なし\u3000R', '男なし\u3000ネック

/Users/yut0takagi/.local/share/virtualenvs/dsb_katoh-Ys4zl5rU/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/yut0takagi/.local/share/virtualenvs/dsb_katoh-Ys4zl5rU/lib/python3.11/site-packages/scipy/stats/_morestats.py:3104: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/var/folders/wb/sqvg1jfx4qvbgf2ndb8gt5300000gn/T/ipykernel_73823/3244260895.py:102: RuntimeWarning: divide by zero encountered in scalar divide
  glass_delta = (mean1 - mean2) / math.sqrt(var2)
/var/folders/wb/sqvg1jfx4qvbgf2ndb8gt5300000gn/T/ipykernel_73823/3244260895.py:80: RuntimeWarning: invalid value encountered in scalar divide
  df = (var1/n1 + var2/n2)**2 / ((var1**2)/(n1**2*(n1-1)) + (var2**2)/(n2**2*(n2-1)))
/var/folders/wb/sqvg1jfx4qvbgf2ndb

In [66]:
for index,v in enumerate(res.keys()):
    print(v)
    sheet_stats.cell(1, 3+index).value = v
wb.save("/Users/yut0takagi/Develop/dsb_katoh/Project1 Metrics_notest (2)のコピー2.xlsx")

test
equal_var_assumed
t
df
p
mean1
mean2
mean_diff
ci_diff
cohens_d
hedges_g
g_ci
glass_delta
n1
n2
var1
var2
levene_p
transform
inverse_means


In [64]:
res.keys()

dict_keys(['test', 'equal_var_assumed', 't', 'df', 'p', 'mean1', 'mean2', 'mean_diff', 'ci_diff', 'cohens_d', 'hedges_g', 'g_ci', 'glass_delta', 'n1', 'n2', 'var1', 'var2', 'levene_p', 'transform', 'inverse_means'])